In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import json
import os

In [ ]:
mask = np.zeros([img.shape[0], img.shape[1], 1], dtype=np.uint8)

for contour in contours:
    contour = np.array(contour)
    xpoints = contour.transpose()[0][0]
    ypoints = contour.transpose()[1][0]
    
    rr, cc = skimage.draw.polygon(xpoints, ypoints)
    
    mask[cc, rr, 0] = img[cc, rr, 0]
    
cv.imshow('asdf', mask)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
img = cv2.imread("datasets/test_mask/mask_Object1_3.png")
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(imgray, 127, 255, 0)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

all_xpoints = []
all_ypoints = []

for contour in contours:
    xpoints = contour.transpose()[0][0]
    ypoints = contour.transpose()[1][0]
    
    all_xpoints.append(xpoints)
    all_ypoints.append(ypoints)

# cv2.imshow('test', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [9]:
def find_contours(img):
    # Grayscale image 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)

    # Find the contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
    xpoints = np.array(contours[0]).transpose()[0]
    ypoints = np.array(contours[0]).transpose()[1]

    return xpoints, ypoints

In [10]:
# class Mask(object):
#     def __init__(self, file):
#         self.name = file[0].split('-')[1].split('.')[0]
#         self.classification = self.name.split('_')[0]
#         self.contours = find_contours(cv2.imread(file[0]))


class Image(object):
    def __init__(self, file):
        print(file)
        self.image = cv2.imread(file)
        idx = len(file.split('/'))
        self.name = file.split('/')[idx-1]
        self.classification = self.name.split('-')[1].split('_')[0]
        self.contours = find_contours(cv2.imread(file))
#         self.mask = self.get_mask()

#     def get_mask(self):
#         path = "./datasets/test_mask/mask_Object" + self.name.split('_Object')[1]
#         return Mask(glob.glob(path))

In [11]:
# Save the points to a json file
images = []

for file in glob.glob("./datasets/test_image/*.png"):
    images.append(Image(file))
    
data = {}

for image in images:
    region = {}

    x_points, y_points = image.contours
    
    region['all_points_x'] = x_points.tolist()
    region['all_points_y'] = y_points.tolist()
    
    image_information = {}
    height, width, _ = image.image.shape
    image_information['filename'] = image.name
    image_information['class'] = image.classification
    image_information['width'] = width
    image_information['height'] = height
    image_information['region'] = region
    
    data['image_' + image.name] = image_information
    
with open('datasets/test_image/annotations.json', 'w') as json_file:
    json.dump(data, json_file)

./datasets/test_image/image-1_0.png


In [ ]:
#For testing
import random

annotations = json.load(open(os.path.join("./val_images/annotations.json")))
annotations = list(annotations.values())

annotation = annotations[0]

fin_img = cv2.imread("./val_images/" + annotation['filename'])

x_points = annotation['region']['all_points_x']
y_points = annotation['region']['all_points_y']
# print(len(y_points) == len(x_points))

color = random.choice([[0,0,255], [255, 0, 0], [0, 255, 0], [255, 255, 0], [0, 255, 255], [255, 0, 255]])

points = np.array([x_points, y_points]).transpose()

cv2.polylines(fin_img, [points], True, color, 2)

# Show
cv2.imshow(annotation['filename'], fin_img)

cv2.waitKey(0)
cv2.destroyAllWindows()